The Goal: Find out if Olist's system (the code) is "guessing" when a product will arrive, or if it's actually "calculating" it based on reality.

In [12]:
import pandas as pd

# Load only the two tables we discussed
orders = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv')
items = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv')


Note: 

What is shipping_limit_date?

This is the most important "System" column in the whole dataset.

Think of it as the Seller's Deadline.

When a customer buys something, the Olist System (the tech) calculates a date and says to the Seller: "You must hand this package over to the courier by this time."

If the seller hands it over after this date, they are "Late" in the eyes of the platform.

So, Connect the "Promise" (Order table) to the "Constraint" (Items table):


In [13]:
# We just need to know the deadline the system gave the seller 

df = pd.merge(orders, items[['order_id', 'shipping_limit_date']], on='order_id')

In [14]:
cols = ['order_purchase_timestamp', 'order_estimated_delivery_date', 'order_delivered_customer_date', 'shipping_limit_date', 'order_delivered_carrier_date']

for col in cols:
    df[col] = pd.to_datetime(df[col])

In [15]:
# How many days does the system give the seller to pack? 

df['system_buffer_days'] = (df['shipping_limit_date'] - df['order_purchase_timestamp']).dt.days


# Tech notes:
# The .dt.days part extracts only the number of whole days from that Timedelta.
# So:
# 3 days → 3
# 2 days 18 hours → still 2
# Anything less than 24 hours → 0



# Let's see the most common buffers

df['system_buffer_days'].value_counts().head(10)


# Tech notes:
# What this line does:
# value_counts() counts how often each buffer value appears
# .head(10) shows the top 10 most frequent ones
# So you’ll get something like:
# 2 days → 18,000 orders
# 3 days → 12,500 orders
# 1 day → 9,200 orders
# …and so on.

system_buffer_days
6     37598
4     19009
7     14774
5     11535
8      6828
2      4739
9      4118
10     2874
3      2814
11     1828
Name: count, dtype: int64

In [16]:
# How long did the seller ACTUALLY take?
# Note: we need the 'order_delivered_carrier_date' (when the carrier got it)
df['actual_seller_handover_days'] = (df['order_delivered_carrier_date'] - df['order_purchase_timestamp']).dt.days

# Now, let's create a 'Late Handover' flag
df['is_seller_late'] = df['actual_seller_handover_days'] > df['system_buffer_days']

# What % of sellers are actually missing that 6-day deadline?
# df['is_seller_late'].value_counts(normalize=True)

In [17]:
# Create a copy so we don't mess up the original df - also, we are doing this, since '>' was failing because of possible null values 
clean_df = df.dropna(subset=['order_delivered_carrier_date', 'shipping_limit_date']).copy()

# Recalculate
clean_df['actual_seller_handover_days'] = (clean_df['order_delivered_carrier_date'] - clean_df['order_purchase_timestamp']).dt.days
clean_df['is_seller_late'] = clean_df['actual_seller_handover_days'] > clean_df['system_buffer_days']

# What % of sellers are actually missing that 6-day deadline?
clean_df['is_seller_late'].value_counts(normalize=True)

is_seller_late
False    0.944678
True     0.055322
Name: proportion, dtype: float64

TECH NOTE: Carrier Delivery Time Overview

This section compares expected vs actual carrier delivery durations to
understand delivery performance and data sanity.

Code:
    df[['actual_carrier_days', 'expected_carrier_days']].describe()

What this does:
- Selects the numeric columns:
    - actual_carrier_days: days taken by the carrier in reality
    - expected_carrier_days: days promised by the system/carrier
- Generates descriptive statistics for both columns:
    count, mean, std, min, 25%, 50% (median), 75%, max

Why this is useful:
- Validates data quality (missing values, negative or zero-day deliveries)
- Compares promise vs performance at a high level
- Identifies variability and extreme delays before deeper analysis

Interpretation tips:
- mean(actual) > mean(expected) → systemic delivery delays
- std(actual) >> std(expected) → high unpredictability in delivery
- max(actual) >> max(expected) → edge-case failures or logistics issues
- median is often more reliable than mean for skewed delivery data

In [18]:
# How long did the CARRIER take?

df['actual_carrier_days'] = (df['order_delivered_customer_date'] - df['order_delivered_carrier_date']).dt.days


# How long did the TECH SYSTEM think the carrier would take? 
# (Total promised time - Buffer given to seller)
df['expected_carrier_days'] = (df['order_estimated_delivery_date'] - df['shipping_limit_date']).dt.days

# Let's compare them
print(df[['actual_carrier_days', 'expected_carrier_days']].describe())

       actual_carrier_days  expected_carrier_days
count        110195.000000          112650.000000
mean              8.733890              16.738589
std               8.611538               9.850839
min             -17.000000            -914.000000
25%               4.000000              12.000000
50%               7.000000              16.000000
75%              11.000000              21.000000
max             205.000000             147.000000


In [19]:
# We only want to look at orders that actually moved forward in time
df_clean = df[(df['actual_carrier_days'] >= 0) & (df['expected_carrier_days'] >= 0)].copy()

# Now run the describe again on this 'Physical' version
print(df_clean[['actual_carrier_days', 'expected_carrier_days']].describe())

       actual_carrier_days  expected_carrier_days
count        109791.000000          109791.000000
mean              8.759516              16.823829
std               8.613627               8.055516
min               0.000000               0.000000
25%               4.000000              12.000000
50%               7.000000              16.000000
75%              11.000000              21.000000
max             205.000000             147.000000


In [20]:
# 1. Filter for orders that actually have all timestamps
df_clean = df.dropna(subset=['order_delivered_customer_date', 'order_delivered_carrier_date']).copy()

# 2. Re-calculate days on the clean data
df_clean['actual_carrier_days'] = (df_clean['order_delivered_customer_date'] - df_clean['order_delivered_carrier_date']).dt.days
df_clean['expected_carrier_days'] = (df_clean['order_estimated_delivery_date'] - df_clean['shipping_limit_date']).dt.days

# 3. Remove the 'Time Travelers' (Negative days caused by data errors)
df_clean = df_clean[(df_clean['actual_carrier_days'] >= 0) & (df_clean['expected_carrier_days'] >= 0)]

# 4. Now, run the describe
print(df_clean[['actual_carrier_days', 'expected_carrier_days']].describe())

       actual_carrier_days  expected_carrier_days
count        109791.000000          109791.000000
mean              8.759516              16.823829
std               8.613627               8.055516
min               0.000000               0.000000
25%               4.000000              12.000000
50%               7.000000              16.000000
75%              11.000000              21.000000
max             205.000000             147.000000


The "Double Padding": The system (Expected) has a mean of 16.8 days, while the physical world (Actual) has a mean of 8.7 days. The system is promising almost exactly 2x the time it actually takes.

Consistency: Look at the std (Standard Deviation). They are almost the same (~8 days). This tells us that the "chaos" or "variability" in the real world is being mirrored by the system, but the system just shifts the whole timeline forward by 8 days to be "safe."

The 75% Mark: Even for the slower 25% of your shipments, they arrive in 11 days, yet your system is still telling those customers it will take 21 days.

In [21]:
# 1. Load the customer mapping file
customers = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv')

# 2. Merge it into our clean analysis dataframe
# We only need the customer_id to link them and the state to group them
df_clean = pd.merge(df_clean, customers[['customer_id', 'customer_state']], on='customer_id', how='left')

# 3. Now let's see the Truth by Geography
state_results = df_clean.groupby('customer_state').agg({
    'actual_carrier_days': 'mean',
    'expected_carrier_days': 'mean'
}).reset_index()

# Calculate the 'System Gap' (Padding)
state_results['system_gap'] = state_results['expected_carrier_days'] - state_results['actual_carrier_days']

# Show the results
print(state_results.sort_values('system_gap'))

   customer_state  actual_carrier_days  expected_carrier_days  system_gap
1              AL            20.362998              25.299766    4.936768
9              MA            17.351690              23.133917    5.782228
24             SE            17.341333              23.520000    6.178667
7              ES            11.746966              18.457079    6.710112
4              BA            15.403204              22.386370    6.983166
11             MS            11.881628              18.889026    7.007398
5              CE            17.161290              24.401122    7.239832
25             SP             5.087755              12.391634    7.303879
16             PI            15.695985              23.164436    7.468451
23             SC            11.134263              18.679873    7.545610
8              GO            11.840580              19.850681    8.010101
18             RJ            11.327036              19.378617    8.051581
26             TO            13.561290

The Better Logic: The "Service Level" Approach

In [22]:
# Calculate mean and std deviation per state
smart_logic = df_clean.groupby('customer_state')['actual_carrier_days'].agg(['mean', 'std']).reset_index()

# Let's see how much 'Safety' we need if we want to be 97% on time (K=2)
smart_logic['suggested_eta'] = smart_logic['mean'] + (2 * smart_logic['std'])

# Compare our 'Smart ETA' to Olist's 'Expected ETA'
comparison = pd.merge(smart_logic, state_results[['customer_state', 'expected_carrier_days']], on='customer_state')
print(comparison[['customer_state', 'mean', 'std', 'suggested_eta', 'expected_carrier_days']].sort_values('mean'))

   customer_state       mean        std  suggested_eta  expected_carrier_days
25             SP   5.087755   5.581069      16.249893              12.391634
17             PR   8.158753   5.967390      20.093532              17.724132
10             MG   8.277485   5.975707      20.228898              17.582416
6              DF   9.227273   6.222446      21.672164              17.500850
23             SC  11.134263   7.796477      26.727216              18.679873
18             RJ  11.327036  10.419806      32.166647              19.378617
22             RS  11.451118   8.588852      28.628822              21.489473
7              ES  11.746966   9.660062      31.067090              18.457079
8              GO  11.840580   8.886047      29.612674              19.850681
11             MS  11.881628   6.902552      25.686732              18.889026
26             TO  13.561290   6.939017      27.439324              21.722581
12             MT  14.338803   7.788018      29.914838          